Задача: посчитать для региона средние температуры в месяц.

Если в одном регионе N станций, то среднее значение - среднее арифметическое по этим N станциям

In [1]:
import pandas as pd

Я скачала данные о станциях из сводного файла в csv, берём только нужные столбцы

In [2]:
stations = pd.read_csv("stations.csv") 
stations = stations[["Код станции", "Код региона"]]
# фрагмент 
stations.head()

,Код станции,Код региона
0,20046,29
1,20069,24
2,20087,24
3,20107,0
4,20289,24


Теперь данные с сайта. Они в формате txt

In [3]:
temperature = pd.read_fwf('wr40708.txt', header = None)
temperature.columns = ['code_station', 'year', 'july', 'august']
temperature.head()

,code_station,year,july,august
0,20046,2008,NaN,NaN
1,20046,2009,0.6,0.2
2,20046,2010,0.5,NaN
3,20046,2011,0.6,0.6
4,20046,2012,1.0,0.9


In [4]:
rain = pd.read_fwf("rain_data\wr41430.txt", header = None)
rain.columns = ['code_station', 'year', 'july', 'august']
rain

,code_station,year,july,august
0,20046,2008,NaN,NaN
1,20046,2009,33.6,10.6
2,20046,2010,8.4,NaN
3,20046,2011,11.8,6.9
4,20046,2012,20.1,16.1
...,...,...,...,...
5480,37663,2014,3.6,5.9
5481,37663,2015,19.6,27.6
5482,37663,2016,72.1,12.7
5483,37663,2017,12.3,0.0


Добавим столбец с кодом региона


In [5]:
rain = pd.merge(rain, stations, left_on=['code_station'], right_on=['Код станции'], how='left')
rain = rain[['Код региона', 'code_station', 'year', 'july', 'august']]
rain = rain.rename({'Код региона': 'code_region'}, axis='columns')
rain

,code_region,code_station,year,july,august
0,29,20046,2008,NaN,NaN
1,29,20046,2009,33.6,10.6
2,29,20046,2010,8.4,NaN
3,29,20046,2011,11.8,6.9
4,29,20046,2012,20.1,16.1
...,...,...,...,...,...
5480,5,37663,2014,3.6,5.9
5481,5,37663,2015,19.6,27.6
5482,5,37663,2016,72.1,12.7
5483,5,37663,2017,12.3,0.0


In [6]:
temperature = pd.merge(temperature, stations, left_on=['code_station'], right_on=['Код станции'], how='left')
temperature = temperature[['Код региона', 'code_station', 'year', 'july', 'august']]
temperature = temperature.rename({'Код региона': 'code_region'}, axis='columns')
temperature

,code_region,code_station,year,july,august
0,29,20046,2008,NaN,NaN
1,29,20046,2009,0.6,0.2
2,29,20046,2010,0.5,NaN
3,29,20046,2011,0.6,0.6
4,29,20046,2012,1.0,0.9
...,...,...,...,...,...
5479,5,37663,2014,21.2,23.2
5480,5,37663,2015,21.3,20.8
5481,5,37663,2016,19.8,22.3
5482,5,37663,2017,22.1,22.3


In [7]:
mean_temperatures = temperature.groupby(['code_region', 'year']).mean()
mean_temperatures = mean_temperatures.drop(columns=['code_station']) # удаляем столбец, потому что средний номер станции нам не сдался
#mean_temperatures = mean_temperatures.drop(['0'])
mean_temperatures

july     august
code_region year                      
0           2008   5.600000   4.600000
            2009   7.100000   6.100000
            2010   6.000000   4.400000
            2011   6.500000   6.200000
            2012   6.000000   5.100000
...                     ...        ...
89          2014  11.800000  10.944444
            2015  13.791667  10.133333
            2016  19.637500  13.677778
            2017  15.977778  11.475000
            2018  15.562500  11.650000

[858 rows x 2 columns]

In [8]:
mean_rain = rain.groupby(['code_region', 'year']).mean()
mean_rain = mean_rain.drop(columns=['code_station'])
mean_rain

july     august
code_region year                      
0           2008  24.800000  23.000000
            2009  13.400000  18.900000
            2010  12.700000  11.500000
            2011  28.100000  20.100000
            2012  32.400000  31.300000
...                     ...        ...
89          2014  64.745455  42.200000
            2015  97.483333  40.516667
            2016  19.762500  45.888889
            2017  13.411111  70.237500
            2018  25.600000  51.762500

[858 rows x 2 columns]

Из года в год может меняться количество станций

In [9]:
mean_temperatures.to_csv('mean_temperatures.csv')

In [10]:
pd.read_csv("mean_temperatures.csv") # проверка

,code_region,year,july,august
0,0,2008,5.600000,4.600000
1,0,2009,7.100000,6.100000
2,0,2010,6.000000,4.400000
3,0,2011,6.500000,6.200000
4,0,2012,6.000000,5.100000
...,...,...,...,...
853,89,2014,11.800000,10.944444
854,89,2015,13.791667,10.133333
855,89,2016,19.637500,13.677778
856,89,2017,15.977778,11.475000


Теперь надо привести в нужный вид

In [12]:
regions = pd.read_csv("for_template.csv")
regions = regions[['Код региона', 'Регион']]
regions = regions.drop([0])
regions.head()

,Код региона,Регион
1,22.0,Алтайский край
2,28.0,Амурская область
3,29.0,Архангельская область
4,30.0,Астраханская область
5,31.0,Белгородская область


In [13]:
# июль
july_data = mean_temperatures['july']
july_data = july_data.unstack(level=-1)
august_data = mean_temperatures['august']
august_data = august_data.unstack(level=-1)
august_data.head()

year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
code_region,,,,,,,,,,,
0,4.600000,6.100000,4.400000,6.200000,5.100000,6.600000,5.800000,6.700000,5.300000,5.400000,6.100000
2,18.300000,15.912500,20.862500,15.637500,19.562500,17.662500,18.650000,14.387500,22.462500,17.900000,16.512500
3,13.521053,14.015789,13.668421,15.168421,13.221053,15.372222,14.700000,17.000000,15.205556,15.077778,15.861111
4,13.620000,14.075000,13.100000,13.460000,13.440000,13.680000,14.080000,13.880000,13.080000,13.560000,14.060000
5,22.500000,18.683333,23.350000,21.016667,23.314286,20.928571,24.033333,22.557143,24.114286,24.057143,21.328571


In [14]:
july_data = pd.merge(regions, july_data, left_on=['Код региона'], right_on=['code_region'], how='left')
august_data = pd.merge(regions, august_data, left_on=['Код региона'], right_on=['code_region'], how='left')
august_data.head()

,Код региона,Регион,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,22.0,Алтайский край,17.275000,17.000000,17.675000,16.900000,18.2625,17.90,18.257143,17.733333,17.171429,17.2625,17.5625
1,28.0,Амурская область,14.775000,14.675000,15.075000,15.925000,12.6000,14.80,16.550000,17.533333,13.550000,15.9500,15.3750
2,29.0,Архангельская область,11.672727,11.633333,14.780000,11.308333,11.7000,15.46,16.030000,11.141667,15.990000,15.3600,15.1800
3,30.0,Астраханская область,25.450000,21.600000,27.950000,24.950000,26.2500,23.75,27.600000,24.450000,27.800000,26.9000,24.0500
4,31.0,Белгородская область,20.700000,17.200000,24.433333,19.300000,20.2000,19.80,20.966667,19.800000,21.266667,21.5000,20.4000


In [18]:
july_data.to_excel('july_mean_temperatures.xlsx', index = False, encoding = 'utf-8')
august_data.to_excel('august_mean_temperatures.xlsx', index = False, encoding = 'utf-8')

In [16]:
# то же самое но с осадками
july_data = mean_rain['july']
july_data = july_data.unstack(level=-1)
august_data = mean_rain['august']
august_data = august_data.unstack(level=-1)
august_data.head()

year,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
code_region,,,,,,,,,,,
0,23.000000,18.900000,11.500000,20.100000,31.300000,98.400000,19.300000,68.100000,38.100000,47.400000,55.200000
2,55.062500,53.637500,40.962500,49.137500,54.687500,56.787500,54.662500,58.387500,27.612500,25.337500,55.712500
3,47.568421,56.778947,52.626316,51.621053,60.157895,36.461111,46.027778,37.727778,41.416667,44.122222,41.022222
4,67.540000,32.975000,49.080000,67.680000,44.840000,54.360000,33.900000,53.040000,44.260000,56.260000,50.080000
5,26.916667,56.583333,8.400000,65.566667,28.414286,44.471429,16.266667,24.242857,11.414286,16.400000,48.214286


In [17]:
july_data = pd.merge(regions, july_data, left_on=['Код региона'], right_on=['code_region'], how='left')
august_data = pd.merge(regions, august_data, left_on=['Код региона'], right_on=['code_region'], how='left')
august_data.head()

,Код региона,Регион,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,22.0,Алтайский край,36.512500,37.175000,15.937500,36.837500,46.187500,74.525000,53.900000,41.116667,38.042857,44.575,34.075
1,28.0,Амурская область,27.725000,39.300000,29.575000,63.325000,43.250000,67.125000,50.750000,36.925000,87.150000,35.825,41.775
2,29.0,Архангельская область,36.463636,32.208333,60.070000,52.650000,55.891667,50.130000,62.830000,45.916667,56.330000,65.020,40.620
3,30.0,Астраханская область,5.200000,45.200000,1.100000,13.000000,10.350000,15.200000,7.450000,5.750000,2.850000,1.850,5.800
4,31.0,Белгородская область,23.000000,17.566667,17.833333,32.833333,61.033333,44.933333,43.333333,11.433333,70.233333,29.700,3.900


In [18]:
july_data.to_excel('july_mean_rain.xlsx', index = False, encoding = 'utf-8')
august_data.to_excel('august_mean_rain.xlsx', index = False, encoding = 'utf-8')